# LMDI Decomposition Demo

This notebook demonstrates the **LMDI decomposition methodology** for explaining
booking changes between two time periods.

## Split Mix Effects

The mix effect is decomposed into two components:

- **Customer Mix Effect**: Impact of changes in customer segment distribution
- **Offer Comp Mix Effect**: Impact of changes in offer competitiveness within segments

### 7 Effects

1. Volume Effect
2. Customer Mix Effect
3. Offer Comp Mix Effect
4. Straight Approval Effect
5. Conditional Approval Effect
6. Straight Booking Effect
7. Conditional Booking Effect

## Setup

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import LMDI calculator and visualization engine
import lmdi_decomposition_calculator as lmdi_calc
import visualization_engine as viz

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 100

## Load Data

In [ ]:
# Load 2D monthly data
data_path = Path.cwd().parent / 'data' / 'funnel_data_2d_mock_with_nonfinanced.csv'
df_monthly = pd.read_csv(data_path)
df_monthly['month_begin_date'] = pd.to_datetime(df_monthly['month_begin_date'])

print(f"Loaded {len(df_monthly)} rows of monthly data")
print(f"\nLenders: {', '.join(sorted(df_monthly['lender'].unique()))}")
print(f"Date range: {df_monthly['month_begin_date'].min().date()} to {df_monthly['month_begin_date'].max().date()}")
print(f"Unique months: {df_monthly['month_begin_date'].nunique()}")

# Show sample
print("\nSample data:")
df_monthly.head(6)

---

# 1. LMDI Decomposition (Single Lender)

Calculate LMDI decomposition for a year-over-year comparison.

In [ ]:
# Calculate LMDI decomposition for June 2023 -> June 2024 (YoY)
results = lmdi_calc.calculate_decomposition(
    df=df_monthly,
    date_a='2023-06-01',
    date_b='2024-06-01',
    lender='ACA'
)

print("LMDI Decomposition Results:")
print(f"  Method: {results.metadata['method']}")
print(f"  Period 1: {results.metadata['date_a']}")
print(f"  Period 2: {results.metadata['date_b']}")
print(f"  Period 1 bookings: {results.metadata['period_1_total_bookings']:,.0f}")
print(f"  Period 2 bookings: {results.metadata['period_2_total_bookings']:,.0f}")
print(f"  Delta bookings: {results.metadata['delta_total_bookings']:+,.0f}")

In [ ]:
# View summary - 7 effects with split mix
print("\nLMDI Decomposition Summary:")
print("="*60)
results.summary

### Split Mix Effects

The decomposition includes:
- **customer_mix_effect**: Impact of changes in customer segment distribution
- **offer_comp_mix_effect**: Impact of changes in offer competitiveness within segments

In [ ]:
# Create waterfall grid visualization
fig_waterfall = viz.create_waterfall_grid(
    summary=results.summary,
    segment_detail=results.segment_detail,
    lender='ACA'
)

plt.show()

---

# 2. Dimension Drilldowns

Drill down into each dimension to see effect contributions.

In [ ]:
# Customer Segment drilldown
fig_segment = viz.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='customer_segment',
    lender='ACA'
)

plt.show()

In [ ]:
# Offer Comp Tier drilldown
fig_tier = viz.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='offer_comp_tier',
    lender='ACA'
)

plt.show()

---

# 3. Segment-Level Detail

The calculator provides segment-level breakdown of effects.

In [ ]:
# Show segment-level data with split mix columns
print("Segment-Level Data:")
print("="*80)

results.segment_detail[[
    'customer_segment', 'offer_comp_tier',
    'period_1_pct_of_total', 'period_1_customer_share', 'period_1_offer_comp_share',
    'period_2_pct_of_total', 'period_2_customer_share', 'period_2_offer_comp_share'
]].head(9)

In [ ]:
# Show split mix effects by segment
print("\nEffects by Segment:")
print("="*80)

results.segment_detail[[
    'customer_segment', 'offer_comp_tier',
    'customer_mix_effect', 'offer_comp_mix_effect', 'total_effect'
]]

---

# 4. Multi-Lender Analysis

In [ ]:
# Verify perfect reconciliation
total_calculated = results.segment_detail['total_effect'].sum()
actual_change = results.metadata['delta_total_bookings']

print("Reconciliation Check:")
print("="*60)
print(f"Calculated total effect: {total_calculated:+,.2f}")
print(f"Actual booking change:   {actual_change:+,.2f}")
print(f"Difference:              {abs(total_calculated - actual_change):.2e}")
print("="*60)

---

# 5. Multi-Lender Analysis

In [ ]:
# Calculate multi-lender decomposition
results_multi = lmdi_calc.calculate_multi_lender_decomposition(
    df=df_monthly,
    date_a='2023-06-01',
    date_b='2024-06-01'
)

print("Multi-Lender LMDI Decomposition Results:")
print(f"  Method: {results_multi.metadata['method']}")
print(f"  Lenders analyzed: {', '.join(results_multi.metadata['lenders'])}")
print(f"\nAggregate Results (All Lenders):")
print(f"  Period 1 bookings: {results_multi.metadata['aggregate_period_1_bookings']:,.0f}")
print(f"  Period 2 bookings: {results_multi.metadata['aggregate_period_2_bookings']:,.0f}")
print(f"  Delta bookings: {results_multi.metadata['aggregate_delta_bookings']:+,.0f}")

In [ ]:
# View aggregate summary
print("\nAggregate Summary (All Lenders):")
results_multi.aggregate_summary

In [ ]:
# Create lender waterfall grid
fig_multi = viz.create_lender_waterfall_grid(
    lender_summaries=results_multi.lender_summaries,
    aggregate_summary=results_multi.aggregate_summary,
    metadata=results_multi.metadata
)

plt.show()

---

# Summary

## 7 Effects

| Effect | Description |
|--------|-------------|
| Volume Effect | Impact from total application volume change |
| Customer Mix Effect | Impact from customer segment distribution change |
| Offer Comp Mix Effect | Impact from offer competitiveness mix change |
| Str Approval Effect | Impact from straight approval rate change |
| Cond Approval Effect | Impact from conditional approval rate change |
| Str Booking Effect | Impact from straight booking rate change |
| Cond Booking Effect | Impact from conditional booking rate change |

## Perfect Reconciliation

All effects sum exactly to the actual booking change.